In [2]:
from ds4400_final_project.dataset.constants import DATASET_FOLDER
from pathlib import Path
from typing import Tuple, Dict
import torch
import torch.nn as nn
from sklearn.preprocessing import normalize, OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np

In [3]:
def load_data_from_file(csv_filename: str) -> Tuple[np.array, np.array, Dict[int, str], Dict[str, int]]:
	""" Load the CSV file from the dataset folder. """
	file = str(Path(DATASET_FOLDER) / csv_filename)
	features_list = np.genfromtxt(file, dtype=None, encoding=None, delimiter=",", skip_header=1, usecols=range(2, 60))
	features = np.array([list(x) for x in features_list])

	# Create a mapping between a numeric value and genre
	index_genre_map = {i: genre for i, genre in enumerate(np.unique(features[:,-1]))}
	genre_index_map = {value: key for key, value in index_genre_map.items()}

	# split the inputs and their labels
	x = normalize(features[:,:57])
	# y = normalize(np.array([genre_index_map[genre] for genre in features[:,-1]]).reshape(-1, 1))

	# take labels and make vector where the value at the index is 1 and everything else is 0
	def make_one_hot(idx):
		v = np.ones((10,))
		v[idx] = 1.0
		return v
	y = np.array([make_one_hot(genre_index_map[genre]) for genre in features[:,-1]])

	return x, y, index_genre_map, genre_index_map

In [4]:
# import the data from the 3 seconds and 30 seconds features CSV
X_3, y_3, index_genre_map_3, genre_index_map_3 = load_data_from_file("features_3_sec.csv")
X_30, y_30, index_genre_map_30, genre_index_map_30 = load_data_from_file("features_30_sec.csv")

print(X_3.shape)
print(X_30.shape)
print(y_3.shape)
print(y_30.shape)

(9990, 57)
(1000, 57)
(9990, 10)
(1000, 10)


In [5]:
# split all the data into training and testing sets
x_3_train, x_3_test, y_3_train, y_3_test = map(torch.FloatTensor, train_test_split(X_3, y_3, test_size=0.33, random_state=42))
x_30_train, x_30_test, y_30_train, y_30_test = map(torch.FloatTensor, train_test_split(X_30, y_30, test_size=0.33, random_state=42))

In [6]:
class ShallowModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(57, 16)
        self.fc2 = nn.Linear(16, 1)
        

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.softmax(x, dim=1)

In [7]:
model = ShallowModel()
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.01)

In [8]:
model.train()
EPOCHS = 1000

for epoch in range(EPOCHS):

    optimizer.zero_grad()

    y_pred = model(x_3_train)

    loss = loss_function(y_pred.squeeze(), y_3_train)

    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))

    loss.backward()
    optimizer.step()

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
model.eval()
y_pred = model(x_3_test)
after_train = loss_function(y_pred.squeeze(), y_3_test) 
print('Test loss after Training' , after_train.item())

Test loss after Training 0.6086820960044861
